## 语料库数据

In [1]:
dir='./data/shakespeare_input.txt'
with open(dir,'r',encoding='utf-8') as f:
    text=f.read()


In [3]:
print(len(text))#整个语料库大小

1115394


In [5]:
#here are the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(f'vocabulary size:{vocab_size}')



 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocabulary size:65


## 分词，训练集/验证集划分

### 分词
这里用的是最简单的分词-字符级标记器（character-level tokenizer），创建两个字典用于存储映射关系。第一个字典存放字符->索引下标映射；第二个字典存放索引下标->字符映射。



##### 拓展知识点
在造分词器的时候，可以权衡codebook size（词汇表）和序列长度。
可以拥有词汇量非常小的非常长的整数序列，也可以拥有词汇量非常大的短整数序列。
？？？
1. 词汇量非常小的非常长的整数序列：

这指的是使用一个较小的词汇表，但允许序列（即文本中的单词或标记序列）有较长的长度。在这种情况下，每个词或标记可能由一个较大的整数来表示，因为词汇表中的项较少，所以可以使用较大的数值范围。
这种方法可能适用于某些特定的应用场景，比如当文本数据具有高度的重复性，或者当模型需要处理非常长的文本序列时。
2. 词汇量非常大的短整数序列：

这指的是使用一个较大的词汇表，但限制序列的长度。在这种情况下，每个词或标记由一个较小的整数来表示，因为词汇表中的项较多，所以每个项的表示范围较小。
这种方法可能更适用于处理多样化的文本数据，因为它能够捕捉到更多的词汇细节，但同时也需要考虑到模型的内存和计算效率，因为较大的词汇表可能会增加模型的复杂度。
--------------
个人理解

如果词汇表很大的话，那么（极端情况下）每个词/字符/字节都会有对应的索引下标。这样会造成一句话的序列可能会很长，所以一般当文本很短的时候，我们可以用大词汇表，小序列长度的方式，增加对词汇细微差别的理解。例如：社交媒体文本通常包含大量的俚语、表情符号和个性化词汇。在这种情况下，可能需要一个较大的词汇表来捕捉这些多样化的表达方式，但由于内存和计算资源的限制，序列长度可能需要被限制。

如果词汇表很小的话，序列长度很长的话。往往是用于一些重复率比较高的长文本。例如：生物信息学，
在处理基因序列数据时，序列长度可以非常长，但使用的“词汇”（即核苷酸）只有四种（A、T、C、G）。这里，序列长度是关键，而词汇量非常小。

--------------
Q:如果在特定场景应用了不恰当的分词策略，会造成什么影响？

Q:对话大模型在实际应用中会根据用户输入的字符长度来动态选择分词策略吗？

In [14]:
###Tokenizers
# create a mapping from characters to integers
str2idx = {ch: i for i, ch in enumerate(chars)}
idx2str = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [str2idx[c] for c in s]
decode = lambda l: ''.join(idx2str[c] for c in l)

In [13]:
print(encode('hii there'))
print(decode(encode('hii there')))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


在实践中，类似的分词策略例如：Google利用SentencePiece，即一种子词类型的标记器。该标记器没有对整个单词进行编码，也没有对单个字符进行编码。它是一个子字单元级别。

OpenAI用了一个称为TickToken的库，它使用字节对编码标记器

In [20]:
#将整个数据集进行编码
import torch
data = torch.tensor(encode(text),dtype=torch.long)#这两个参数是什么意思？#这个函数在干嘛？
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([ 0, 18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43,
        44, 53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52,
        63,  1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,
         1, 57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39,
        49,  6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15,
        47, 58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50,
        50,  1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1,
        58, 53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51,
        47, 57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43,
        42,  8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57,
        58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1,
        63, 53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56,
      

### 划分训练集/验证集

In [21]:
n = int(0.9*len(data))
train_data = data[:n]#训练集
val_data = data[n:]#验证集

In [23]:
print(len(train_data))
print(len(val_data))

1003854
111540


## 数据加载器：批量数据块